# Standard Randomized Benchmarking

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

**Standard Randomized Benchmarking (SRB)** is the algorithm for benchmarking the full set of Clifford gates by a single parameter, **Error Per Clifford (EPC)**, using randomization techniques.
This tutorial introduces **SRB**, covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).


## Theory
### Clifford gates
The Clifford Gate is a restricted class of gates - the gates which transform elements of the Pauli group into other elements of the Pauli group[1]

$$
Cl(P_n)\equiv \left\{U\vert UP_nU^{\dagger}=P_n\right\}.
$$

The basic Clifford Gate has {H, S, CZ}. (or {H, S, T, CNOT}, equivalently.) Clifford circuit that constructed by Clifford gates can be effectively simulated by classical computer in polynomial complexity (Gottesman-Knill theorem)[1]. Therefore it chose to be used in SRB and is widely used in quantum error correction. We realize this function in qep to obtain $n$-qubit Clifford Gate.

In [ ]:
from qcompute_qep.quantum.clifford import Clifford,random_clifford
from qcompute_qep.benchmarking.standardrb import StandardRB

In [ ]:
n = 2 # number of qubits
cliff = Clifford(n)
print(cliff) # print the Clifford Gate in elementary gate form

As shown above, it is a two-qubit Clifford gate, see this paper[2] for more details of Clifford gate construction.

### Random circuits

The structure of random circuit in SRB was as followed:

![random-clifford-circuit](figures/srb-clifford-circuit.png "Figure 1. random Clifford circuit")

Experimental implementation of random circuit. We first apply $m$ random Clifford gates,
then we apply the inverse of whole $m$ Clifford gates $C_{m+1}$

$$
C_{m+1} = \left(C_1C_2\cdots C_{m-1}C_{m}\right)^{-1}.
$$

For the ideal situation, the output state should be equal to init state

$$
C_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert \phi \rangle = \vert \phi \rangle.
$$

### Randomized benchmarking

However, The result will not the same in noisy quantum device. As the number of quantum gates increases (that is, the depth of the circuit increases), the noise will accumulates and amplifies. We have such intuition that if the quantum gates provided by a quantum device have good performance and small error, the cumulative error will be relatively small even if the number of gates increases. We can measure the error of different depth of circuit and fit the data to the following equation to calculate the average Clifford gate quality:

$$
p(m) = Af^m+B,
$$

where $p(m)$ is $\textrm{Tr}\left[OC_{m+1}C_{m}C_{m-1}\cdots C_2C_1\vert\phi\rangle\!\langle\phi\vert\right],$ which is observation result of each circuits, $m$ is depths, $f$ is fidelity, and $A$ and $B$ absorb state preparation and measurement errors (SPAM). We demonstrate this by using the following code.

## Practice


In [ ]:
import QCompute

# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian

### Single qubit circuit

![single-qubit](figures/srb-single-clifford-circuit.png "Figure 2: single qubit circuit")

In the following code, we construct and measure 20 random circuit in each depths and average the result. Run the example and see the final result

In [ ]:
single_qubits = [0]
srb = StandardRB()
single_qubit_results = srb.benchmark(qubits=single_qubits,
                          qc=qc,
                          repeats=20,
                          shots=200,
                          seq_lengths=[1, 5, 10, 15, 20, 25])
srb.plot_results()

### Two qubit circuit

The circuit as shown following

![double-qubit](figures/srb-two-clifford-circuit.png "Figure 3: two-qubit circuit")

In [ ]:
# 2 qubit SRB
two_qubits = [0,1]
srb2 = StandardRB()
two_qubits_result = srb2.benchmark(qubits=two_qubits,
                          qc=qc,
                          repeats=20,
                          shots=200,
                          seq_lengths=[1, 2, 3, 4, 5, 6]
                          )
srb2.plot_results()

As shown above, we obtain the curve and the parameters from fitting procedure.
The equation for **EPC** calculating is:

$$
E = \frac{d-1}{d}(1-f),
$$

where $d = 2^N$.
We can print the result of parameters.

In [ ]:
print('The fidelity of 1 qubit SRB is :',single_qubit_results['f'])
print('The Error Per Clifford of 1 qubit SRB is:',single_qubit_results['epc'])
print('The fidelity of 2 qubit SRB is :',two_qubits_result['f'])
print('The Error Per Clifford of 2 qubit SRB  is:',two_qubits_result['epc'])

We have finished the **SRB** and successfully obtained the **EPC** of noisy quantum device. More information of **SRB** can be found on this paper[3].

## References
\[1\] Gottesman, Daniel. "The Heisenberg representation of quantum computers." [arXiv](https://arxiv.org/abs/quant-ph/9807006) preprint quant-ph/9807006 (1998).

\[2\] Selinger, Peter. "Generators and relations for n-qubit Clifford operators." [Logical Methods in Computer Science](https://arxiv.org/abs/1310.6813v4) 11 (2015).

\[3\] Helsen, Jonas, et al. "A general framework for randomized benchmarking." [arXiv](https://arxiv.org/abs/2010.07974) preprint arXiv:2010.07974 (2020)